Starter kernel using TensorFlow to train a model (EfficientNet) using K-fold cross validation. Weights and Biases is used for experiment tracking. 

Idea:

* Each data point consist of 6 spectrograms where the first, third and fifth spectrograms can potentially have a UFO signal. 
* Stack these spectrograms to get a three channel image followed by resizing to 224x224 resolution.
* Train an EfficientNet-B0 as a baseline. 

The model was trained on a single V100 GPU (GCP instance). 

> I will update the kernel with the CV score and LB score along with W&B dashboard to show how the training progressed. 

| Method        | CV Score           | LB Score  | W&B Dashboard |
| ------------- |:-------------:| :-----:| -------: |
| Single Model     | 0.95 | 0.91 | [W&B Run Page](https://wandb.ai/ayush-thakur/kaggle-seti/runs/314vuphr) |
| 5 Fold training  | 0.93      |   0.93 | [W&B Project Page](https://wandb.ai/ayush-thakur/kaggle-seti)

I am publisizing this kernel so that I can be of use if you are biased towards using TensorFlow/Keras. If you want to train the baseline on a Kaggle kernel consider reducing the batch size. 

# 🧰 Imports and Setups

In [ ]:
!pip install -q --upgrade wandb 

import wandb
print(wandb.__version__)
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_addons as tfa
from tensorflow.keras import mixed_precision

import os
import gc
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# 📀 Hyperparameters

In [ ]:
TRAIN_PATH = 'train/'
AUTOTUNE = tf.data.experimental.AUTOTUNE

CONFIG = dict (
    seed = 42,
    num_labels = 1,
    num_folds = 5,
    train_val_split = 0.2,
    img_width = 224,
    img_height = 224,
    batch_size = 64,
    epochs = 100,
    learning_rate = 4e-4,
    wandb_kernel = True,
    architecture = "CNN",
    infra = "Kaggle",
)

# 🔨 Build Input Pipeline

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print(f'Number of train images: {len(df)}')
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
df.head()

In [ ]:
class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(df['target'].values),
                                    y=df['target'].values)

class_weights_dict = {key: val for key, val in zip(np.unique(df['target'].values), class_weights)}
class_weights_dict                                                            

In [ ]:
Fold = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
for n, (train_index, val_index) in enumerate(Fold.split(df, df['target'])):
    df.loc[val_index, 'fold'] = int(n)
df['fold'] = df['fold'].astype(int)
df.groupby(['fold', 'target']).size()

In [ ]:
def load_npy(path):
    # load npy data
    data = np.load(path.numpy()).astype(np.float32)
    # stack 
    data = np.dstack((data[0], data[2], data[4]))
    # Normalize
    
    return data

@tf.function
def load_resize_spec(df_dict):
    # Load image
    [image,] = tf.py_function(load_npy, [df_dict['img_path']], [tf.float32])
    image.set_shape((273, 256, 3))
    
    # Resize image
    image = tf.image.resize(image, (CONFIG['img_height'], CONFIG['img_width']))
    
    # Parse label
    label = df_dict['target']
    label = tf.cast(label, tf.float32)
    
    return image, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def get_dataloaders(train_df, valid_df):
    trainloader = tf.data.Dataset.from_tensor_slices(dict(train_df))
    validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))

    trainloader = (
        trainloader
        .shuffle(1024)
        .map(load_resize_spec, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )

    validloader = (
        validloader
        .map(load_resize_spec, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )
    
    return trainloader, validloader

# 🐤 Model

In [ ]:
def get_model():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = True

    inputs = layers.Input((CONFIG['img_height'], CONFIG['img_width'], 3))
    x = base_model(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(CONFIG['num_labels'])(x)
    outputs = layers.Activation('sigmoid', dtype='float32', name='predictions')(outputs)
    
    return models.Model(inputs, outputs)

tf.keras.backend.clear_session() 
model = get_model()
model.summary()

In [ ]:
CONFIG['model_name'] = 'efficientnetb0-folds'
CONFIG['group'] = 'K-Fold-EnetB0'
CONFIG['run_name'] = 'baseline-k-fold'

# Callbacks

In [ ]:
# Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='min',
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=CONFIG['learning_rate'])

# 🚄 Train

In [ ]:
def get_predictions(model, validloader, valid_df):
    y_pred = []
    for image_batch, label_batch in validloader:
        preds = model.predict(image_batch)
        y_pred.extend(preds)
        
    valid_df['preds'] = y_pred
    
    return valid_df 

In [ ]:
oof_df = pd.DataFrame()

for fold in range(CONFIG['num_folds']):
    # Prepare train and valid df
    train_df = df.loc[df.fold != fold].reset_index(drop=True)
    valid_df = df.loc[df.fold == fold].reset_index(drop=True)

    # Prepare dataloaders
    trainloader, validloader = get_dataloaders(train_df, valid_df)
    
    
    # Initialize model
    tf.keras.backend.clear_session()
    model = get_model()

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate'])
    model.compile(optimizer, 
                  loss=tfa.losses.SigmoidFocalCrossEntropy(), 
                  metrics=[tf.keras.metrics.AUC(curve='ROC')])


    # Update CONFIG dict with the name of the model.
    print('Training configuration: ', CONFIG)

    # Initialize W&B run
    run = wandb.init(project='kaggle-seti', 
                     config=CONFIG,
                     group=CONFIG['group'], 
                     job_type='train',
                     name=CONFIG['run_name'])

    # Train
    _ = model.fit(trainloader, 
                  epochs=CONFIG['epochs'],
                  validation_data=validloader,
                  class_weight=class_weights_dict,
                  callbacks=[WandbCallback(),
                             earlystopper,
                             reduce_lr])
    
    
    # Evaluate
    loss, auc = model.evaluate(validloader)
    wandb.log({'Val AUC-ROC': auc})
    
    # Save model
    model_name = CONFIG['model_name']
    MODEL_PATH = f'models/{model_name}'
    os.makedirs(MODEL_PATH, exist_ok=True)
    count_models = len(os.listdir(MODEL_PATH))
    
    model.save(f'{MODEL_PATH}/{model_name}_{count_models}.h5')

    # Get Prediction on validation set
    _oof_df = get_predictions(model, validloader, valid_df)
    oof_df = pd.concat([oof_df, _oof_df])

    # Close W&B run
    run.finish()
    
    del model, trainloader, validloader, _oof_df
    _ = gc.collect()

![img](https://i.imgur.com/QW7uCAK.gif)

In [ ]:
def correct_preds(row):
    return row.preds[0]

oof_df['preds'] = oof_df.apply(lambda row: correct_preds(row), axis=1)

In [ ]:
metric = tf.keras.metrics.AUC()
metric.update_state(oof_df.target.values, oof_df.preds.values)
print(f'CV Score: {metric.result().numpy()}')